In [1]:
using ProgressMeter
using JLD
using Random
include("../alpcahus.jl")
include("../helperFunctions.jl")
include("../cluster.jl")

hungarianAlgorithm (generic function with 1 method)

In [2]:
D = 50 # ambient space dimension
d = 3 # subspace dimension
goodPoints = 6 # good points per subspace
ν1 = 0.1 # noise variance good points
K = 2 # subspace count
varianceRatio = range( 1,300,5) # variance ratio for heatmaps
pointRatio = range(1,50,5) # point ratio between good bad data for heatmaps
trials = 100; # smoother results
qValue = 50; # 24 old value

In [3]:
seedVector1 = rand(Xoshiro(1234), 1:100000, trials*length(varianceRatio)*length(pointRatio));
seedVector2 = rand(Xoshiro(4321), 1:100000, trials*length(varianceRatio)*length(pointRatio));
seedVector3 = rand(Xoshiro(2143), 1:100000, trials*length(varianceRatio)*length(pointRatio));
seedVector4 = rand(Xoshiro(3412), 1:100000, trials*length(varianceRatio)*length(pointRatio));

In [4]:
# # SIMPLE TEST
# ν2 = varianceRatio[end]*ν1
# badPoints = round(Int, pointRatio[end]*goodPoints)
# U1 = generateSubspace(D, d; seed=seedVector1[10]) 
# U2 = generateSubspace(D, d; seed=seedVector2[20])
# X1 = generateData(U1, [ν1,ν2], [goodPoints,badPoints]; seed=seedVector3[30])
# X2 = generateData(U2, [ν1,ν2], [goodPoints,badPoints]; seed=seedVector4[700])
# X = hcat(X1,X2);
# trueCluster = vec([1*ones(size(X1)[2],1); 2*ones(size(X2)[2],1)]);
# estimatedCluster = ALPCAHUS(X,K, [d,d]; B=1, q=10, T=100, alpcahIter=10, subspaceMethod=:alpcah, finalStep=true, spectralStart=true) 
# clusterError(trueCluster, estimatedCluster)

In [5]:
ekssTensor = zeros(length(varianceRatio), length(pointRatio),trials)
alpcahTensor = zeros(length(varianceRatio), length(pointRatio),trials)
oracleTensor = zeros(length(varianceRatio), length(pointRatio),trials)

count = 1
@showprogress 1 "Running..." for i = 1:length(varianceRatio) # varianceRatio
     for j = 1:length(pointRatio) # prepend Threads.@threads for multi-threading
        for k = 1:trials
            # other parameters
            ν2 = varianceRatio[i]*ν1
            badPoints = round(Int, pointRatio[j]*goodPoints)
            # subspace generation
            U1 = generateSubspace(D, d; seed=seedVector1[count]) 
            U2 = generateSubspace(D, d; seed=seedVector2[count])
            # data generation
            X1 = generateData(U1, [ν1,ν2], [goodPoints,badPoints]; seed=seedVector3[count])
            X2 = generateData(U2, [ν1,ν2], [goodPoints,badPoints]; seed=seedVector4[count])
            # complete data matrix
            X = hcat(X1,X2);
            # cluster assignments
            trueCluster = vec([1*ones(size(X1)[2],1); 2*ones(size(X2)[2],1)]);
            # ekss trial
            estimatedCluster = ALPCAHUS(X,K, [d,d]; B=1, q=qValue, T=100, subspaceMethod=:pca, finalStep=false, spectralStart=true)
            ekssTensor[i,j,k] = clusterError(trueCluster, estimatedCluster)
            # alpcahus trail
            estimatedCluster = ALPCAHUS(X,K, [d,d]; B=1, q=qValue, T=100, alpcahIter=10, subspaceMethod=:alpcah, finalStep=true, spectralStart=true) 
            alpcahTensor[i,j,k] = clusterError(trueCluster, estimatedCluster)
            # noisy oracle trial       
            noisyU1 = svd(X1[:,1:goodPoints]).U[:,1:d]
            noisyU2 = svd(X2[:,1:goodPoints]).U[:,1:d]
            estimatedCluster = 1 .+ Vector{Int8}(norm.(eachcol(noisyU1'*X)) .> norm.(eachcol(noisyU2'*X)));
            oracleTensor[i,j,k] = clusterError(trueCluster, estimatedCluster)
            # increase count
            count = count + 1
        end
    end
end

save("trialData_q50.jld", "ekss", ekssTensor, "alpcah", alpcahTensor, "oracle", oracleTensor);

Running... 100%|█████████████████████████████████████████| Time: 0:17:53


In [7]:
trialData = load("kss/trialData.jld")
ekssTensor = trialData["ekss"]
alpcahTensor = trialData["alpcah"]
oracleTensor = trialData["oracle"];

In [9]:
generateHeatmap(pointRatio, varianceRatio, ekssTensor; plotType=:median, methodType="KSS", ν1=ν1, goodPoints=goodPoints)
savefig("kss_median.pdf")

"/home/javier/Desktop/alpcahus/smart_kss/kss_median.pdf"

In [10]:
generateHeatmap(pointRatio, varianceRatio, alpcahTensor; plotType=:median, methodType="ALPCAHUS", ν1=ν1, goodPoints=goodPoints)
savefig("alpcahus_median.pdf")

"/home/javier/Desktop/alpcahus/smart_kss/alpcahus_median.pdf"

In [11]:
generateHeatmap(pointRatio, varianceRatio, oracleTensor; plotType=:median, methodType="Noisy Oracle", ν1=ν1, goodPoints=goodPoints)
savefig("oracle_median.pdf")

"/home/javier/Desktop/alpcahus/smart_kss/oracle_median.pdf"

In [12]:
generateHeatmap(pointRatio, varianceRatio, ekssTensor; plotType=:mean, methodType="KSS", ν1=ν1, goodPoints=goodPoints)
savefig("kss_mean.pdf")

"/home/javier/Desktop/alpcahus/smart_kss/kss_mean.pdf"

In [13]:
generateHeatmap(pointRatio, varianceRatio, alpcahTensor; plotType=:mean, methodType="ALPCAHUS", ν1=ν1, goodPoints=goodPoints)
savefig("alpcahus_mean.pdf")

"/home/javier/Desktop/alpcahus/smart_kss/alpcahus_mean.pdf"

In [14]:
generateHeatmap(pointRatio, varianceRatio, oracleTensor; plotType=:mean, methodType="Noisy Oracle", ν1=ν1, goodPoints=goodPoints)
savefig("oracle_mean.pdf")

"/home/javier/Desktop/alpcahus/smart_kss/oracle_mean.pdf"

In [15]:
generateHeatmap(pointRatio, varianceRatio, ekssTensor; plotType=:variance, methodType="KSS", ν1=ν1, goodPoints=goodPoints)
savefig("kss_variance.pdf")

"/home/javier/Desktop/alpcahus/smart_kss/kss_variance.pdf"

In [16]:
generateHeatmap(pointRatio, varianceRatio, alpcahTensor; plotType=:variance, methodType="ALPCAHUS", ν1=ν1, goodPoints=goodPoints)
savefig("alpcahus_variance.pdf")

"/home/javier/Desktop/alpcahus/smart_kss/alpcahus_variance.pdf"

In [17]:
generateHeatmap(pointRatio, varianceRatio, oracleTensor; plotType=:variance, methodType="Noisy Oracle", ν1=ν1, goodPoints=goodPoints)
savefig("oracle_variance.pdf")

"/home/javier/Desktop/alpcahus/smart_kss/oracle_variance.pdf"